In [1]:
import os
import csv
import cv2
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing
import matplotlib.pyplot as plt
import random
import pandas as pd
import re
import pickle
from PIL import Image
from keras import losses

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier

# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder, OneHotEncoder


# Visualizations will be shown in the notebook.
%matplotlib inline

Using TensorFlow backend.


In [2]:
# augment functions

# transform x y direction, input original rgb image
def transform_image(image):
    trans_range = 4
    tx = trans_range*(np.random.uniform()-0.5)
    ty = 2*(np.random.uniform()-0.5)
    rows,cols,_ = image.shape
    M = np.float32([[1,0,tx],[0,1,ty]])
    image = cv2.warpAffine(image,M,(cols,rows))
    return image

# Horizontal Flip
def flip_image(image):
    # Horizontal Flip
    image = cv2.flip(image, 1)
    return image

# adjust brightness
def bright_image(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv)
    v = v.astype(np.float)
#     v *= (np.random.uniform()+0.2)
    v *= np.random.choice([0,8, 1, 1.2])
    v[v>255] = 255
    v[v<0] = 0
    v = v.astype(np.uint8)
    final_hsv = cv2.merge((h, s, v))
    image_hsv = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2RGB)
    return image_hsv

# warp image
def warp_image(img):
    rows,cols,_ = img.shape
    # random scaling coefficients
    rndx = np.random.rand(3) - 0.5
    rndx *= cols * 0.06   # this coefficient determines the degree of warping
    rndy = np.random.rand(3) - 0.5
    rndy *= rows * 0.06
    # 3 starting points for transform, 1/4 way from edges
    x1 = cols/4
    x2 = 3*cols/4
    y1 = rows/4
    y2 = 3*rows/4

    pts1 = np.float32([[y1,x1],
                       [y2,x1],
                       [y1,x2]])
    pts2 = np.float32([[y1+rndy[0],x1+rndx[0]],
                       [y2+rndy[1],x1+rndx[1]],
                       [y1+rndy[2],x2+rndx[2]]])

    M = cv2.getAffineTransform(pts1,pts2)

    dst = cv2.warpAffine(img,M,(cols,rows))
       
    return dst

# input rgb image
def preprocess_image(image):
    rows,cols,_ = image.shape
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
#     image = cv2.equalizeHist(image)
#     image = image[50:rows-20, 0:cols]
#     image = cv2.resize(image, (64, 64), interpolation=cv2.INTER_AREA)
    image = image/255.0 - 0.5
#     image = np.expand_dims(image, axis=2)
    return image

In [3]:
images_car = glob.glob('../vehicle_data/vehicles/*/*.png')
images_nocar = glob.glob('../vehicle_data/non-vehicles/*/*.png')
cars = []
notcars = []

for image in images_car:
    cars.append(image)
for image in images_nocar:
    notcars.append(image)
    
X_ = np.hstack((cars, notcars))
y_ = np.hstack((np.ones(len(cars)), np.zeros(len(notcars))))
print(X_.shape)
print(y_.shape)

(17760,)
(17760,)


In [4]:
labelencoder_y_1 = LabelEncoder()
y_ = labelencoder_y_1.fit_transform(y_)

X_, y_ = shuffle(X_, y_)

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.1, random_state=rand_state)
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

15984
1776
15984
1776


In [5]:
def generator_samples(X_, y_, batch_size=32, if_train=False):
    num_samples = len(X_)
    while True: # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_samples = X_[offset:offset+batch_size]
            batch_indexes = y_[offset:offset+batch_size]
            images = []
            indexes = []
            for batch_sample in zip(batch_samples, batch_indexes):
                name = batch_sample[0]
#                 center_image = cv2.imread(name)
                image = np.asarray(Image.open(name))
                index = float(batch_sample[1])
                if if_train:
                    image = transform_image(image)
                    if np.random.choice([True, False]):
                        image = flip_image(image)
                    image = bright_image(image)
                    image = warp_image(image)
                image = preprocess_image(image)
                images.append(image)
                indexes.append(index)

            # trim image to only see section with road
            X_ = np.array(images)
            y_ = np.array(indexes)
            yield sklearn.utils.shuffle(X_, y_)

            
# compile and train the model using the generator function
train_generator = generator_samples(X_train, y_train, 32, True)
validation_generator = generator_samples(X_test, y_test, 32, False)

In [6]:
import pickle
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.models import Sequential, Model
from keras.layers import Lambda, Concatenate
from keras.layers import Cropping2D, Input
from keras.layers import Reshape
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from keras import backend as K
from keras.optimizers import SGD, Adam, RMSprop
import tensorflow
import cv2

lamda_w = 1e-4

# model = Sequential()
input1 = Input(shape=(64, 64, 3))
         
conv1 = Conv2D(16, (1, 1), strides=(1, 1), padding='valid', kernel_regularizer=regularizers.l2(lamda_w))(input1)
conv1 = Activation('relu')(conv1)
conv1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(conv1)

conv2 = Conv2D(32, (5, 5), strides=(1, 1), padding='valid', kernel_regularizer=regularizers.l2(lamda_w))(conv1)
conv2 = Activation('relu')(conv2)
conv2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(conv2)

conv3 = Conv2D(32, (1, 1), strides=(1, 1), padding='valid', kernel_regularizer=regularizers.l2(lamda_w))(conv2)
conv3 = Activation('relu')(conv3)

conv4 = Conv2D(64, (5, 5), strides=(1, 1), padding='valid', kernel_regularizer=regularizers.l2(lamda_w))(conv3)
conv4 = Activation('relu')(conv4)
conv4 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(conv4)

conv5 = Conv2D(64, (1, 1), strides=(1, 1), padding='valid', kernel_regularizer=regularizers.l2(lamda_w))(conv4)
conv5 = Activation('relu')(conv5)

conv6 = Conv2D(128, (5, 5), strides=(1, 1), padding='valid', kernel_regularizer=regularizers.l2(lamda_w))(conv5)
conv6 = Activation('relu')(conv6)

conv7 = Conv2D(1, (1, 1), strides=(1, 1), padding='valid', kernel_regularizer=regularizers.l2(lamda_w))(conv6)
conv7 = Activation('relu')(conv7)

output = conv7

model = Model(inputs=input1, outputs=output)

adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss=losses.sparse_categorical_crossentropy , optimizer=adam)
model.summary()

filepath="model/model-{epoch:02d}-{val_loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# model.load_weights('model-09-0.0200.hdf5')

history_object = model.fit_generator(train_generator, samples_per_epoch= len(X_train), \
                                     validation_data=validation_generator, \
                                     nb_val_samples=len(X_test), \
                                     nb_epoch=3, verbose=1, callbacks=callbacks_list)
model.save('model.h5') 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        12832     
_________________________________________________________________
activation_2 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
__________

/home/tzyhpcom/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:59: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/tzyhpcom/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:59: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., verbose=1, callbacks=[<keras.ca..., steps_per_epoch=15984, epochs=3, validation_steps=1776)`


Epoch 1/3


ValueError: Error when checking target: expected activation_7 to have 4 dimensions, but got array with shape (32, 1)